In [431]:
import requests
import json
import re
import csv
import io

from math import *
from lxml import etree
from io import StringIO, BytesIO
from datetime import date,datetime,timedelta

from isodate import parse_duration

from oscar_schedules import Schedule , number_expected

In [432]:
BASE_URL = 'https://oscar.wmo.int/surface/rest/api'
SEARCH_API = BASE_URL + '/search/station?'
XML_DOWNLOAD_URL = BASE_URL + "/wmd/download/" 

In [433]:
# mapping of country and regiona names to codes
cmap=json.load(open("country-map.json"))
rmap=json.load(open("region-map.json"))

In [434]:
def deg_min_sec(degrees = 0.0,long=False):
    if type(degrees) != 'float':
        try:
            degrees = float(degrees)
        except:
            print( '\nERROR: Could not convert %s to float.' %(type(degrees)))
            return 0
    minutes = degrees%1.0*60
    seconds = minutes%1.0*60
    if not long:
        n = "S" if degrees < 0 else "N"
    else:
        n = "W" if degrees < 0  else "E"
    
    return "{deg} {mint:02d} {sec:02d}{n}".format( deg= int(floor(degrees)), mint=int(floor(minutes)), sec=int(seconds), n=n)

def extractInfoFromXML(xml,varid):
    namespaces = { 
    'wmdr':'http://def.wmo.int/wmdr/2017',
    'gml' : 'http://www.opengis.net/gml/3.2',        
    'xlink' : 'http://www.w3.org/1999/xlink',
    'xsi' : 'http://www.w3.org/2001/XMLSchema-instance',
    'om' : 'http://www.opengis.net/om/2.0'
    }
    
    # rather complicated xpath to identify pressure deployments that are still open
    xpath_deployment = '/wmdr:WIGOSMetadataRecord/wmdr:facility/wmdr:ObservingFacility/wmdr:observation/wmdr:ObservingCapability/wmdr:observation/om:OM_Observation[ om:observedProperty[@xlink:href="http://codes.wmo.int/wmdr/'+varid+'"] ]/om:procedure/wmdr:Process/wmdr:deployment/wmdr:Deployment[ (wmdr:validPeriod/gml:TimePeriod[ not( boolean(gml:endPosition/text()) ) ]  ) and ( wmdr:dataGeneration/wmdr:DataGeneration/wmdr:reporting/wmdr:Reporting/wmdr:internationalExchange = "true"  ) ]'
    # relative xpath to get the instrument height 
    xpath_instrument_pos = 'wmdr:deployedEquipment/wmdr:Equipment/wmdr:geospatialLocation/wmdr:GeospatialLocation/wmdr:geoLocation/gml:Point/gml:pos/text()'
    # relative xpath to get the schedule 
    xpath_schedule = 'wmdr:dataGeneration/wmdr:DataGeneration/wmdr:schedule/wmdr:Schedule'
    # relative xpath to reporting interval
    xpath_rep = 'wmdr:dataGeneration/wmdr:DataGeneration/wmdr:reporting/wmdr:Reporting'
    
    
    depl = xml.xpath(xpath_deployment,namespaces=namespaces)
    
    ret = {}
    if len(depl) == 0 :
        return ret
    
    depl = depl[0]

    instrument_pos = depl.xpath(xpath_instrument_pos,namespaces=namespaces)
    if len(instrument_pos)>0 :
        (lat,lon,elevation) = instrument_pos[0].split()
        ret["elevation"]=elevation
        ret["latitude"]=lat
        ret["longitude"]=lon
    
    s= {}
    schedule = depl.xpath(xpath_schedule,namespaces=namespaces)
    if len(schedule) > 0:
        schedule = schedule[0]
        
        prefix = "{"+namespaces["wmdr"]+"}"
        for name in [ "{}{}".format(s,e) for e in ["Month","Weekday","Hour","Minute"] for s in ["start","end"] ]:
            s[name] = schedule.find(prefix + name).text
        
    ret["schedule"] = s

    reporting = depl.xpath(xpath_rep,namespaces=namespaces)
    if len(reporting)>0:
        reporting=reporting[0]
        
        prefix = "{"+namespaces["wmdr"]+"}"
        
        rep_intex = reporting.find(prefix + "internationalExchange")
        ret["schedule"]["international"] = rep_intex.text 
        
        rep_int = reporting.find(prefix + "temporalReportingInterval")
        ret["schedule"]["interval"] = rep_int.text
        
        
    return ret


def makeSchedule(schedule):
    p = {}
    months = {"Month":"month","Weekday":"week","Hour":"hour","Minute":"min"}
    ends = {"start":"from","end":"to"}

    for name,newname in { "{}{}".format(s,e):"{}_{}".format(e2,s2) for e,e2 in months.items() for s,s2 in ends.items() }.items():
        p[newname]=schedule[name]
        
    p["international"] = bool(schedule["international"])
    p["interval"] = parse_duration( schedule["interval"] ).total_seconds()

    s = Schedule(**p)
    print(s)
    return s
            

In [435]:
variables = [216,224, 227, 256, 310, 12000] # from https://codes.wmo.int/wmdr
networks = ['GOS','RBSN','RBSNp','RBSNs','RBSNsp','RBSNst','RBSNt','ANTON','ANTONt']
statuses = ['operational','preOperational','partlyOperational']
# we search for UK stations to keep the result set smaller 
arg = "territoryName=GBR&territoryName=landFixed&ProgramAffiliation={affiliation}&ObservedVariable={variable}"

In [436]:
r = requests.get(SEARCH_API + arg.format( affiliation=",".join(networks) , variable=",".join(  [str(v) for v in variables])  ) )
if r.status_code != 200:
    print("ERROR: status: {}".format(r.status_code))
    sys.exit(1)
    
result = r.json()
print("we have {} results".format(len(result)))

we have 1212 results


In [437]:
result[0]

{'id': 92,
 'name': 'Ascension Island',
 'region': 'Africa',
 'territory': 'United Kingdom (the)',
 'declaredStatus': 'Operational',
 'latitude': -7.9699997902,
 'longitude': -14.3999996185,
 'elevation': 91,
 'stationTypeName': 'Land (fixed)',
 'wigosStationIdentifiers': [{'wigosStationIdentifier': '0-20008-0-ASC',
   'primary': True}],
 'stationTypeId': 1,
 'dateEstablished': '1979-08-26T23:00:00.000+0000',
 'stationStatusCode': 'operational',
 'stationTypeCode': 'landOceanSurface',
 'stationProgramsDeclaredStatuses': 'ESRLCCG:Operational, GAW Regional:Operational, SHADOZ:Operational, TCCON:Operational'}

In [438]:
# extract primary wigos id from result, if there is a primary wigos id
station_infos = {wid['wigosStationIdentifier']:station for station in result if 'wigosStationIdentifiers' in station for wid in station['wigosStationIdentifiers']  }

In [440]:
results = []
count=0
for wid,station in station_infos.items():
    
    if not station["declaredStatus"].lower() in statuses: #skip, since not operational
        print("skipping {name} {declaredStatus}".format(**station))
        continue
  
    url = XML_DOWNLOAD_URL + wid
    print("downloading: ",url)
    res = requests.get(url)
    
  
    if res.status_code == 200:
        
        myxml = res.content
        
        print("checking station {name} id: {id}".format(**station))
         
        # we can get basic elements from the JSON
        region = station['region']
        region_id = rmap[ station['region'].lower() ]
        
        country = station['territory']
        country_id = cmap[ station['territory'].lower() ]
        
        latitude = station['latitude']
        longitude = station['longitude']
        elevation = station['elevation'] if 'elevation' in station else None
        
        internal_id = station['id']
               
        station_id = wid
        
        index = None
        sub_index = None
        name = station['name']
        
        m = re.match('0-2000([0-1])-0-(.*)',station_id)
        
        if m:
            index = m.group(2)
            sub_index = m.group(1)
        
        # some elements need to be obtained from the XML export
        try:
            root = etree.fromstring( myxml )
        except etree.XMLSyntaxError as xmle:
            print("parsing error in {} xml: {} error: {}".format(station["name"], myxml[0:200]  , xmle))
            continue
        
        info_pressure = extractInfoFromXML(root,"216")    
        print(info_pressure)
        sos={}
        if info_pressure:
            pressure_schedule = makeSchedule(info_pressure["schedule"])

            today = date.today()
            for i,hour in enumerate(range(0,24,3)):
                today_dt = datetime( today.year,today.month,today.day,hour )
                lower_boundary = today_dt - timedelta(hours=0) 
                upper_boundary = today_dt + timedelta(hours=3)

                sos["SO-{}".format(i+1)] = number_expected([pressure_schedule,], lower_boundary,upper_boundary )       


        info_pressure_profile = extractInfoFromXML(root,"12000")
        print(info_pressure_profile)
        uas={}
        if info_pressure_profile:
            pressure_profile_schedule = makeSchedule(info_pressure_profile["schedule"])

            for i,hour in enumerate(range(0,24,6)):
                today_dt = datetime( today.year,today.month,today.day,hour )
                lower_boundary = today_dt - timedelta(hours=0) 
                upper_boundary = today_dt + timedelta(hours=6)

                uas["UA-{}".format(i+1)] = number_expected([pressure_profile_schedule,], lower_boundary,upper_boundary )       

            print("{wigosid} {index} {subindex} ".format(wigosid=station_id,index=index,subindex=sub_index))

        hp = info_pressure["elevation"] if (info_pressure and info_pressure["elevation"]) else elevation
        hha = elevation
        
        row = {'RegionId': region_id, 'Region Name':region,'Country Area':country,'Country Code':country_id,
               'StationId':internal_id ,'IndexNbr': index ,'Index SubNbr' : sub_index, 'Station Name': name,
               'Lat' : deg_min_sec( latitude, False), 'Long': deg_min_sec(longitude, True), 
               'Hp' : hp , 'HpFlag' : None, 'Hha':hha, 'HhaFlag':None,
               'PressureDefId':None,
              'SO-1' : None,'SO-2' : None,'SO-3' : None,'SO-4' : None,'SO-5' : None,'SO-6' : None,'SO-7' : None,'SO-8' : None,            
               'ObsHs' : None, 
                'UA-1' : None, 'UA-1' : None, 'UA-3' : None,'UA-4' : None,
               'ObsRems':None}
        
        # set synoptic observations and soundings calculated before. Also calculate ObsHs
        obshs = []
        for key,val in sos.items():
            row[key] = "X" if val>0 else "."
            obshs.append(val)
            
        for key,val in uas.items():
            row[key] = "X" if val>0 else "."
            
        # calculate ObsHs
        cur = None
        start_idx = 0

        l=[]
        for i,t in enumerate([ None if t<3 else "S" if t>=6 else "H" for t in obshs  ]):
            if t != cur:
                if cur:
                    l.append("{}{:02d}-{:02d}".format(cur,start_idx*3,i*3))
                    cur=t
                    start_idx = i
                cur=t
                start_idx=i

        if cur:
            l.append("{}{:02d}-{:02d}".format(cur,start_idx*3,24))
            
        if l:
            row["ObsHs"] = ",".join(l)
        
        
        results.append(row)
        
        
    else:
        print("WARNING: failed to download {wid} {r}".format(wid=wid,r=r))
        
        
    if count>10:
        break
    count=count+1

downloading:  https://oscar.wmo.int/surface/rest/api/wmd/download/0-20008-0-ASC
checking station Ascension Island id: 92
{}
{}
downloading:  https://oscar.wmo.int/surface/rest/api/wmd/download/0-20000-0-61966
checking station DIEGO GARCIA (61966-0) id: 8395
{'elevation': '8.0', 'latitude': '-7.2997222222', 'longitude': '72.4', 'schedule': {'startMonth': '1', 'endMonth': '12', 'startWeekday': '1', 'endWeekday': '7', 'startHour': '0', 'endHour': '23', 'startMinute': '0', 'endMinute': '59', 'international': 'true', 'interval': 'PT30M'}}
Jan-Dec/Mon-Sun/0:0-23:59 0:30:00
{}
downloading:  https://oscar.wmo.int/surface/rest/api/wmd/download/0-20000-0-61967
checking station DIEGO GARCIA (61967-0) id: 8396
{'elevation': '3.0', 'latitude': '-7.3', 'longitude': '72.4', 'schedule': {'startMonth': '1', 'endMonth': '12', 'startWeekday': '1', 'endWeekday': '7', 'startHour': '0', 'endHour': '18', 'startMinute': '0', 'endMinute': '59', 'international': 'true', 'interval': 'PT3H'}}
Jan-Dec/Mon-Sun/0:0-

In [441]:
with io.StringIO() as csvfile:
    if results:
        fieldnames = results[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames,delimiter="\t")
        writer.writeheader()

        for r in results:
            writer.writerow(r)

        print(csvfile.getvalue())
        
        with open("out.txt","w",newline="\n") as out:
            out.write(csvfile.getvalue())

RegionId	Region Name	Country Area	Country Code	StationId	IndexNbr	Index SubNbr	Station Name	Lat	Long	Hp	HpFlag	Hha	HhaFlag	PressureDefId	SO-1	SO-2	SO-3	SO-4	SO-5	SO-6	SO-7	SO-8	ObsHs	UA-1	UA-3	UA-4	ObsRems
RA-I	Africa	United Kingdom (the)	GBR	92			Ascension Island	-8 01 48S	-15 36 00W	91		91															
RA-I	Africa	United Kingdom (the)	GBR	8395	61966	0	DIEGO GARCIA (61966-0)	-8 42 01S	72 24 00E	8.0					X	X	X	X	X	X	X	X	S00-24				
RA-I	Africa	United Kingdom (the)	GBR	8396	61967	0	DIEGO GARCIA (61967-0)	-8 42 00S	72 24 00E	3.0		1			X	X	X	X	X	X	X	.					
RA-I	Africa	United Kingdom (the)	GBR	8392	61903	0	ST. HELENA	-16 03 37S	-6 19 49W	438.0					X	X	X	X	X	X	X	X	S00-24				
RA-I	Africa	United Kingdom (the)	GBR	8390	61901	0	ST. HELENA IS.	-16 03 29S	-6 19 58W	436.0		435			X	X	X	X	X	X	X	X	H00-24				
RA-I	Africa	United Kingdom (the)	GBR	8391	61902	0	WIDE AWAKE FIELD (ASCENSION IS.)	-8 01 48S	-15 35 39W	79.0		86			X	X	X	X	X	X	X	X					
RA-VII	Antarctica	United Kingdom (the)	GBR	12488	89013	0	BAL